# 🧠 딥러닝 기반 이미지 분류 (불량/정상 이미지)
사전 학습된 모델과 이미지 증강을 활용한 고도화된 분류 모델 구현

## 📁 Step 1: Google Drive 연동 및 데이터 업로드

In [ ]:
import gdown
import zipfile
import os

file_id = "1rIHf4kA8DDlfuD2s9zltMt6SMULBiwtJ"
gdown.download(f"https://drive.google.com/uc?id={file_id}", quiet=False)

## 📦 Step 2: 이미지 ZIP 파일 압축 해제

In [ ]:
zip_path = "/content/noodle.zip"

extract_path = "noodle"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"압축 해제 완료")

## 🧹 Step 3: 이미지 전처리 + 증강

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = val_datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_gen.class_indices.keys())
print("클래스:", class_names)


## 🧠 Step 4: 사전 학습된 모델 (MobileNetV2) 로드 및 미세 조정

In [ ]:

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # 초기엔 학습 비활성화

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 🔁 Step 5: 기본 학습 (기존 가중치 고정)

In [ ]:

history = model.fit(train_gen, validation_data=val_gen, epochs=5)


## 🧬 Step 6: 상위 층 미세 조정 (Fine-tuning)

In [ ]:

base_model.trainable = True

# 일부 상위층만 학습 가능하도록 설정
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # 낮은 학습률
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(train_gen, validation_data=val_gen, epochs=5)


## 📊 Step 7: 정확도 및 손실 시각화

In [ ]:

import matplotlib.pyplot as plt

# 기본 학습 결과
plt.plot(history.history['accuracy'], label='Train Acc (Base)')
plt.plot(history.history['val_accuracy'], label='Val Acc (Base)')

# 미세 조정 결과
plt.plot(fine_tune_history.history['accuracy'], label='Train Acc (FT)')
plt.plot(fine_tune_history.history['val_accuracy'], label='Val Acc (FT)')

plt.title("Model Accuracy")
plt.legend()
plt.show()


## 🔍 Step 8: 개별 이미지 예측 및 시각화

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# 예측을 위한 샘플 이미지 로드
sample_path = val_gen.filepaths[0]  # 검증 데이터 중 하나
img = image.load_img(sample_path, target_size=img_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 예측 수행
pred = model.predict(img_array)
pred_class = class_names[np.argmax(pred)]

# 시각화
plt.imshow(img)
plt.title(f"predict: {pred_class}")
plt.axis('off')
plt.show()


## 📉 Step 9: Confusion Matrix 및 Classification Report

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# 전체 검증 데이터에 대한 예측 수행
val_gen.reset()
predictions = model.predict(val_gen, steps=val_gen.samples // val_gen.batch_size + 1)
y_pred = np.argmax(predictions, axis=1)
y_true = val_gen.classes

# 혼동 행렬 출력
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 분류 리포트 출력
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
